<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class9-1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-435: Class 9.1**

**Includes in-class updates**


**Announcements**
* Code for your Final Project is due in *one week:* Nov. 21!



## My plot style defaults

In [ ]:
from matplotlib import rc

rc('figure', figsize=(12,4))
rc('figure', facecolor='#aaaaaa')     # Better figure background for dark mode

rc('font', family='Liberation Serif') # Nicer font
rc('font', size=20)                   # Larger font size for labels

# Setup
The usual modules...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
from matplotlib import animation, rc
from scipy import signal
import numpy.fft as fft
import pickle
import time
# import matplotlib.patches as patches
# import matplotlib.colors
# import matplotlib.cm

rc('animation', html='jshtml')
rc('animation', embed_limit=2**30)

path = '/content/drive/My Drive/eces435-work/class8.1/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'atari2600'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Helper functions

`myPlot()`

A quick time-domain signal plot function with my default figure settings and a time x-axis (in seconds).
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `N=#` Number of samples to plot (default: length of signal)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Plot format string (default: none)
  * *New* `x_lim=#` or `x_lim=[x1,x2]` Specify the x-axis limit(s) of the plot
  * *New* `y_lim=#` or `y_lim=[y1,y2]` Specificy the y-axis limit(s) of the plot

In [ ]:
def myPlot(sig, N=0, fs=44100, fig_size=(16,4), x_ax=True, y_ax=True, lw=1, fmt='', x_lim=0, y_lim=0):
  if N==0:
    N = len(sig)

  fig = plt.figure(figsize=fig_size)
  t = np.arange(N)/fs

  plt.plot( t[:N], sig[:N], fmt, linewidth=lw)

  plt.xlabel('Time (sec)')
  ax = plt.gca()    # gca(): "Get current axis", the graph object that's currently plotted
  
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = N/fs   # End of input signal
    else:
      x_lim2 = x_lim
    plt.xlim(0, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim != 0:
      plt.ylim(-y_lim, y_lim)
  else:
    plt.ylim(y_lim)

  fig.tight_layout()
  #plt.ion()
  plt.close()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`myPlotFFT()`

Plot the magnitude frequency response (in dB FS) of a signal with my default figure settings and a frequency x-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `n_fft=#` The size of FFT to use (default: length of input signal)
  * `n_win=#` The length of window to use (default: length of input)
  * `win='hann'` The type of window to use (default: `hann`, or `rect`)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `x_lim=# or (#,#)` Frequency axis limits (max or range, in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Change plot formatting (default: none)

In [ ]:
def myPlotFFT(sig, fs=44100, n_fft=0, n_win=0, win='hann', neg_f=False, x_lim=0, y_lim=0, fig_size=(16,6),x_ax=True, y_ax=True, lw=1, fmt=''):
  if n_fft==0:
    n_fft = len(sig)
  if n_win==0:
    n_win = len(sig)

  if win=='hann':  
    win = np.hanning(n_win)
    win_scale = 2
  else:
    win = np.ones(n_win)
    win_scale = 1

  S = np.fft.fft(sig * win, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  if neg_f:
    f = f - (fs/2)
    S = np.fft.fftshift(S)

  S_mag = 2*win_scale*np.abs(S) / n_win     # Frequency magnitude, normalized by length
                                            #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                            #    x2 for Hann because window has 0.5 average

  S_mag += 1e-15                  # Add a small offset to avoid log(0) errors
  S_dBFS = 20*np.log10(S_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w


  fig = plt.figure(figsize=fig_size)
  plt.plot(f, S_dBFS, fmt, linewidth=lw) 
  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = fs/2
    if neg_f:
      x_lim = -fs/2
    plt.xlim(x_lim, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim < 0:
      plt.ylim(y_lim, 0)
    elif y_lim > 0:
      plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB FS)')

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`mySpectrogram()`

A simple wrapper to compute and plot the spectrogram of a signal with my default figure settings, a time x-axis (in seconds) and a frequency y-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `win='window_name'` The type of analysis window to use (default: 'hann')
  * `n_win=#` The length of window to use per frame (default: 1024)
  * `n_fft=#` The size of FFT to use (default: 1024)
  * `x_lim=# or (#,#)` x-axis limit or range (in seconds)
  * `y_lim=# or (#,#)` y-axis limit or range (in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x-ax=True/False` Show x-axis (default: True)
  * `y-ax=True/False` Show y-axis (default: True)

In [ ]:
def mySpectrogram(sig, fs=44100, win='hann', n_win=1024, olap=512, n_fft=1024, x_lim=0, y_lim=0, fig_size=(12,6), x_ax=True, y_ax=True):
  f1, t1, Sxx = signal.stft(sig, fs, window=win, nperseg=n_win, noverlap=olap, nfft=n_fft)

  fig = plt.figure(figsize=fig_size)

  S_mag = 4*np.abs(Sxx) + 1e-15
  S_dBFS = 20*np.log10(S_mag)
  
  plt.pcolormesh(t1, f1, S_dBFS)
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = len(sig) / fs
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim == 0:
      y_lim = fs/2
    plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # plt.ion()
  
  # Returning the figure causes issues with interactive matplotlib
  # return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`zplane()` New function to plot system (filter) information in the $z$-plane.



In [ ]:
def zplane(zeros=[], poles=[], xy_lim=[-1.5,1.5]):
  fig = plt.figure(figsize=(6,6))
  ax = plt.axes()

  # Create radial coordinates as for unit circle contour
  N_pts = 200
  w = np.linspace(0, 2*np.pi, N_pts)
  z_uc = np.exp(1j*w)
  re_uc = np.real(z_uc)
  im_uc = np.imag(z_uc)

  # Draw unit circle and set axis limits
  ax.plot(re_uc, im_uc, 'k:', linewidth=1)
  ax.set_ylim(xy_lim)
  ax.set_xlim(xy_lim)

  # Plot zeros (o) and poles (x)
  re_z = np.real(zeros)
  im_z = np.imag(zeros)
  ax.plot(re_z, im_z, 'bo', markersize=15, fillstyle='none')
  re_p = np.real(poles)
  im_p = np.imag(poles)
  ax.plot(re_p, im_p, 'bx', markersize=13, fillstyle='none')
  
  # X-Y axis labels
  ax.set_xlabel('Real')
  ax.set_ylabel('Imag')

`freqz3D()` A function to visualize the full response across the $z$-plane.

Lots of code here, most of it is to handle the graphics.

In [ ]:
def freqz3D(B=[1], A=[1], azim=-60,
            x_lim=(-1.5,1.5), y_lim=(-1.5,1.5), z_lim=(-30,10),
            x_ticks=(-1,0,1), y_ticks=(-1,0,1),
            fig_size=(14,8), lw=2, xy_spacing=0.005, init_elev=45,
            draw_uc=True, draw_pz=True, draw_contour=True,
            hide_ax=False, hide_z=True, show_fig=True):

  fig = plt.figure(figsize=(14,8))
  ax = plt.axes(projection='3d') 
  z_dB_min = z_lim[0]

  if show_fig==False:
    plt.close()

  # Make cartesian grid in z-plane for freqz surface
  X = np.arange(-1.5, 1.501, xy_spacing) # dB requires different spacing
  Y = np.arange(-1.5, 1.501, xy_spacing) # dB spacing
  X, Y = np.meshgrid(X, Y)
  R = np.sqrt(X**2 + Y**2)
  z = X + Y*1j

  # Create radial coordinates as for unit circle contour(s)
  w = np.linspace(0, 2*np.pi, 200)
  z_c = np.exp(1j*w)
  X_c = np.real(z_c)
  Y_c = np.imag(z_c)

  # If transfer function is provided, plot the freqz surface
  if len(B) > 0:
    Bz =  B[0] * np.ones(np.shape(z))
    Bz_c = B[0] * np.ones(len(z_c))
    for n in range(1,len(B)):
      Bz = Bz + np.divide( B[n], z**n )
      Bz_c = Bz_c + np.divide( B[n], z_c**n )

    if len(A) > 0:
      Az = A[0] * np.ones(np.shape(z))
      Az_c = A[0] * np.ones(len(z_c))
      for n in range(1,len(A)):
        Az = Az + np.divide( A[n], z**n )
        Az_c = Az_c + np.divide( A[n], z_c**n )
    else:
      Az = 1
      Az_c = 1

    Hz = Bz / Az
    Hz_c = Bz_c / Az_c
    Z = np.abs(Hz)
    Z_c = np.abs(Hz_c)
    Z_dB = 20 * np.log10(Z)
    Z_c_dB = 20 * np.log10(Z_c)

    # Plot the freqz surface
    surf = ax.plot_surface(X, Y, Z_dB, color='lightgoldenrodyellow', alpha=0.4, linewidth=0 )
    
  Z_proj = np.zeros(len(w))
  Z_proj_dB = Z_proj + z_lim[0] # include dB offset, depends on z-axis lower limit

  # Draw unit circle at bottom face of z-plane
  if draw_uc:
    ax.plot3D(X_c, Y_c, Z_proj_dB, 'w',linewidth=1)

  # Draw freqz contour on unit circle
  an_object, = ax.plot3D(X_c, Y_c, Z_c_dB, 'y',linewidth=lw)
  if draw_contour == False:
    #Z_c_dB += 200 # Hopefully, +200dB is out of view
    an_object.set_linewidth(0.)

  # Create placeholder animation object for frequency marker
  an_object2, = ax.plot3D([0], [0], [-60], 'w:',linewidth=1)

  if draw_pz:
    if len(B) > 0:
      # Plot pole-zero markers
      zeros,poles,k = signal.tf2zpk(B, A)
      x_z = np.real(zeros)
      y_z = np.imag(zeros)
      x_p = np.real(poles)
      y_p = np.imag(poles)
      z_object, = ax.plot3D(x_z, y_z, z_dB_min*np.ones(len(x_z)), 'wo', markersize=15, fillstyle='none')
      p_object, = ax.plot3D(x_p, y_p, z_dB_min*np.ones(len(x_p)), 'wx', markersize=13, fillstyle='none')
  else:
    # Create empty objects
    z_object, = ax.plot3D([0], [0], [z_dB_min], linewidth=0)
    p_object, = ax.plot3D([0], [0], [z_dB_min], linewidth=0)


  # Set camera view angle
  ax.elev = init_elev
  ax.azim = azim

  # make the panes transparent
  ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
  ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
  ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.05))

  # make the grid lines transparent
  ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
  ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
  ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)

  ax.zaxis.set_alpha(0.5)

  ax.set_zlim(z_lim) #dB
  ax.set_xlim(x_lim)
  ax.set_ylim(y_lim)

  if hide_ax:
    ax.set_axis_off()
  else:
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    ax.tick_params(colors='white')
    ax.tick_params(length=10,width=2,labelsize=20)
    for tick in ax.get_xticklabels():
        tick.set_fontname("Liberation Serif")

    for tick in ax.get_yticklabels():
        tick.set_fontname("Liberation Serif")

    fig.patch.set_facecolor('#111111')    # I think this makes the background dark, for preview in Colab

    #ax.set_frame_on(False)
    ax.set_facecolor('#212121')
    ax.get_xaxis().line.set_color('w')
    ax.get_yaxis().line.set_color('w')

    # X-Y axis labels
    ax.set_xlabel('Re', color='w', size=24, fontname='Liberation Serif')
    ax.xaxis.labelpad = 20
    ax.set_ylabel('Im', color='w', size=24, fontname='Liberation Serif')
    ax.yaxis.labelpad = 20

    if hide_z:
      ax.zaxis.line.set_lw(0.)
      ax.set_zticks([])

  fig.tight_layout()

#  if show_fig:
#    plt.show()

  return fig #, ax, an_object, an_object2, surf, p_object, z_object,

# Improving your CODEC

## Load audio files

In [ ]:
load_path_1 = '/content/drive/MyDrive/eces435-work/Labs/Lab5/data/CoolSong_mono.wav'
load_path_2 = '/content/drive/MyDrive/eces435-work/Labs/Lab5/data/WildSong_mono.wav'
load_path_3 = '/content/drive/MyDrive/eces435-work/class8.1/u2.wav'
paths = [load_path_1,load_path_2,load_path_3]

In [ ]:
y = []
fs = []
dur = 30    # Load first 20 seconds

for p in paths:
  y_i, fs_i = sf.read(p)
  y_i = np.float16(y_i) #This will make sure that the audio you are comparing with for the project is 16 bit
  display(ipd.Audio(y_i[0:fs_i*dur], rate=fs_i))
  y.append(y_i[0:fs_i*dur])
  fs.append(fs_i)

## Reference encoder and decoder

In [ ]:
#@title Reference ENCODER

def encode(y, fs): 
  #Paste Encoder Code here. Inputs and Outputs MUST be the same as seen here.

  f1, t1, X = signal.stft(y, 44100, nperseg=1024, noverlap=512, nfft=1024)
  n_freqs, n_frames = np.shape(X)
  N_c = 60

  X_mag = np.abs(X)
  x_sortIdx = np.argsort(X_mag, axis=0) 

  fft_idx = np.int16(x_sortIdx[-N_c:,:])
  fft_real = np.zeros([N_c, n_frames], dtype='float16')
  fft_imag = np.zeros([N_c, n_frames], dtype='float16')
  
  for n in range(n_frames):
    X_sort = X[fft_idx[:,n], n] 
    fft_real[:,n] = np.real(X_sort)
    fft_imag[:,n] = np.imag(X_sort)

  y_compressed = [fft_idx, fft_real, fft_imag]
  return y_compressed

In [ ]:
#@title Your DECODER

def decode(y_compressed):
  #Paste Decoder Code here. Inputs and Outputs MUST be the same as seen here.

  fft_idx = y_compressed[0]
  fft_real = y_compressed[1]
  fft_imag = y_compressed[2]

  n_freqs, n_frames = np.shape(fft_idx)
  X_c = 1j * np.zeros([513, n_frames])
  
  # f1, t1, X_n = signal.stft(x_n, 44100, nperseg=N_fft, noverlap=N_olap, nfft=N_fft) 

  for n in range(n_frames):
    X_c[fft_idx[:,n],n] = fft_real[:,n] + 1j*fft_imag[:,n]

  t_c, y_reconstructed = signal.istft(X_c, 44100, nperseg=1024, noverlap=512, nfft=1024)

  return y_reconstructed

## Test encoder upgrades

In [ ]:
from sys import ps1
#@title ENCODER upgrades

def encode2(y, fs): 
  #Paste Encoder Code here. Inputs and Outputs MUST be the same as seen here.
  f1, t1, X = signal.stft(y, 44100, nperseg=1024, noverlap=512, nfft=1024)
  n_freqs, n_frames = np.shape(X)
  N_c = 50

  X_out = np.copy(X)
  X_mag = np.abs(X)
  # x_sortIdx = np.argsort(X_mag, axis=0) 

  # fft_idx = np.int16(x_sortIdx[-N_c:,:])
  fft_idx = np.zeros([N_c, n_frames], dtype='int16')
  fft_real = np.zeros([N_c, n_frames], dtype='float16')
  fft_imag = np.zeros([N_c, n_frames], dtype='float16')
  noise_pow = np.zeros([5, n_frames])

  hann513 = np.hanning(513)
  hann257 = np.hanning(257)
  hann129 = np.hanning(129)

  channels = np.zeros([5,513])
  channels[0,:65] = hann129[-65:]
  channels[1,:129] = hann129
  channels[2,64:129] = hann129[:65]
  channels[2,128:257] = hann257[-129:]
  channels[3,128:257] = hann257[:129]
  channels[3,256:] = hann513[-257:]
  channels[4,256:] = hann513[:257]

  for n in range(n_frames):
    # X_sort = X[fft_idx[:,n], n]
    for k in range(N_c): 
      peak_idx = np.argmax(X_mag[:,n])
      fft_idx[k,n] = peak_idx
      fft_real[k,n] = np.real(X[peak_idx,n])
      fft_imag[k,n] = np.imag(X[peak_idx,n])
      # X_mag[peak_idx,n] = 0

      # This is a really bad implementation of Masking
      # Do better than this.
      peak_pm = int( peak_idx / 25 )
      p1 = max(peak_idx - peak_pm, 0)
      p2 = min(peak_idx + peak_pm, 512)
      if p1 == p2:
        X_mag[p1,n] = 0
        X_out[p1,n] = 0
      else:
        X_mag[p1:p2,n] = 0
        X_out[p1:p2,n] = 0
    
    X_n = X_mag[:,n]
    noise_pow[0,n] = np.sum(np.multiply(channels[0,:], X_n**2))
    noise_pow[1,n] = np.sum(np.multiply(channels[1,:], X_n**2))
    noise_pow[2,n] = np.sum(np.multiply(channels[2,:], X_n**2))
    noise_pow[3,n] = np.sum(np.multiply(channels[3,:], X_n**2))
    noise_pow[4,n] = np.sum(np.multiply(channels[4,:], X_n**2))

    if n==2100:
      plt.plot(20*np.log10(X_mag[:,n] + 1e-15))

  y_compressed = [fft_idx, fft_real, fft_imag, noise_pow]
  return y_compressed #, X_out

In [ ]:
temp = np.ones(10)
temp[3:3]

### Residual (noise) channels

#### Uniform channels

In [ ]:
hann257 = np.hanning(257)

channels = np.zeros([5,513])
channels[0,:129] = hann257[-129:]
channels[1,:257] = hann257
channels[2,128:385] = hann257
channels[3,256:] = hann257
channels[4,384:] = hann257[:129]

plt.plot(channels.T)

#### Non-uniform channels (better)

In [ ]:
hann513 = np.hanning(513)
hann257 = np.hanning(257)
hann129 = np.hanning(129)

channels = np.zeros([5,513])

channels[0,:65] = hann129[-65:]

channels[1,:129] = hann129

channels[2,64:129] = hann129[:65]
channels[2,128:257] = hann257[-129:]

channels[3,128:257] = hann257[:129]
channels[3,256:] = hann513[-257:]

channels[4,256:] = hann513[:257]

plt.plot(channels.T)

## Decoder upgrades

In [ ]:
#@title DECODER upgrades

def decode2(y_compressed):
  #Paste Decoder Code here. Inputs and Outputs MUST be the same as seen here.
  fft_idx = y_compressed[0]
  fft_real = y_compressed[1]
  fft_imag = y_compressed[2]

  n_freqs, n_frames = np.shape(fft_idx)
  X_c = 1j * np.zeros([513, n_frames])
  
  for n in range(n_frames):
    X_c[fft_idx[:,n],n] = fft_real[:,n] + 1j*fft_imag[:,n]

  t_c, y_reconstructed = signal.istft(X_c, 44100, nperseg=1024, noverlap=512, nfft=1024)

  return y_reconstructed

## Test Codec

In [ ]:
#@title Original Audio

ipd.Audio(y[2], rate=44100)

In [ ]:
#@title Reference Codec

y_c0 = encode(y[2], 44100)  # Encode the audio sample
y_r0 = decode(y_c0)
ipd.Audio(y_r0, rate=44100)

In [ ]:
#@title Upgraded Codec
y_c2 = encode2(y[2], 44100)
y_r2 = decode2(y_c2)
ipd.Audio(y_r2, rate=44100)
# plt.xlim(0,512)

In [ ]:
#@title Residual power output
noise_pow = y_c2[3]
plt.plot(noise_pow.T)

In [ ]:
plt.pcolormesh(20*np.log10(np.abs(X_out)))

# Evaluation

In [ ]:
#@title Run Codec on Audio Samples
ert_ave = 0
drt_ave = 0

y_compressed = []
y_reconstructed = []
# S_reconstructed = []

for i in [0,1,2]:                       # Loops through the test audio samples
  sample = y[i]
  rate = fs[i]

  encodeStartTime = time.time()         # Starts the execution timer
  y_compressed_i = encode(sample,rate)  # Encode the audio sample
  ert = time.time() - encodeStartTime   # Saves the encoder execution time
  y_compressed.append(y_compressed_i)

  decodeStartTime = time.time()         # Start the execution timer
  y_reconstructed_i = decode(y_compressed_i)  # Decode the compressed audio
  drt = time.time() - decodeStartTime   # Saves the decoder execution time

  y_reconstructed.append(y_reconstructed_i)
  
  ert_ave += ert                        # Sum the encode exectution time
  drt_ave += drt                        # Sum the decode execution time

ert_ave /= 3                            # Average encode execution times
drt_ave /= 3                            # Average decode execution times
print(ert_ave)
print(drt_ave)

In [ ]:
#@title Compute Compression Ratio

def compression(song, song_compressed):
  Size_y = len(pickle.dumps(song))
  print('Size of original file: ', Size_y)
  Size_y_compressed = len(pickle.dumps(song_compressed))
  print('Size of compressed file: ', Size_y_compressed)
  Comp_Ratio = Size_y_compressed / Size_y
  print('Compression ratio: ', Comp_Ratio)
  
  return Comp_Ratio

comp_ave = 0

for i in [0,1,2]:
  Compression_Ratio = compression(y[i], y_compressed[i])
  comp_ave += Compression_Ratio

comp_ave /= 3
print("Average Compression Ratio: ", comp_ave)

In [ ]:
#@title Compute SNR

def SNR(original, output):
    # Normalize
    # original = original/np.max(np.abs(original)) # Normalize to -1 to 1
    # output =  output/np.max(np.abs(output)) # Normalize to -1 to 1
    
    # Find the shorter of the original vs. output
    L = min(len(original), len(output))

    # Compute SNR
    noise = original[:L] - output[:L]
    
    powS = np.mean(original**2)
    powN = np.mean(noise**2)

    snr = powS/powN
    snr = 10*np.log10(snr)
    return snr

snr_ave = 0

for i in [0,1,2]:
  snr = SNR(y[i],y_reconstructed[i])
  print("SNR for compressed audio: ", snr)

  snr_ave += snr

snr_ave /= 3
print("Average SNR: ", snr_ave)

In [ ]:
#@title Display Overall Codec Results
print("Results Averaged over Three Test Samples:")
print("--------------------------------------------")
print("Avg. Compression Ratio")
print(comp_ave)
print()
print("Avg. SNR")
print(snr_ave)
print()
print("Avg. Run Time ")
print("Encode:", ert, "      Decode:", drt)
print()
print("Audio for Perception Test")
display(ipd.Audio(y_reconstructed[0], rate=fs[0]))
display(ipd.Audio(y_reconstructed[1], rate=fs[1]))
display(ipd.Audio(y_reconstructed[2], rate=fs[2]))